# Visual Analysis of the Systemic Risk Index (SRI)

In this notebook, we visually analyze the Systemic Risk Index (SRI) created in the previous step. The goal is to validate its effectiveness by comparing it against historical market events and asset performance. We will examine:
1.  The SRI's behavior during known financial crises and recessions.
2.  The dynamic, time-varying correlation between the SRI and key asset classes (equities, gold, and bonds).

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# Load data including SRI
df = pd.read_csv("../data/systemic_risk_index.csv", index_col=0, parse_dates=True)

## Plot 1: SRI Performance During Market Crises

To validate our Systemic Risk Index, we first plot it against the S&P 500 (SPY). An effective risk index should rise during periods of market stress, which typically coincide with stock market downturns. We have overlaid the official NBER-defined recession periods, along with COVID-19 Breakout (shaded in gray) to provide historical context for major economic crises.

In [ ]:
# Initialize a figure with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the Systemic Risk Index (SRI) trace to the primary y-axis
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["SRI"],
        name="Systemic Risk Index (SRI)",
        line=dict(color="crimson", width=2),
    ),
    secondary_y=False,
)

# Add the S&P 500 (SPY) trace to the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["SPY"],
        name="S&P 500 (SPY Price)",
        line=dict(color="royalblue", width=2, dash="dash"),
    ),
    secondary_y=True,
)

if df["USRECP"].sum() > 0:
    # Find the start of a recession (where USREC goes from 0 to 1)
    recession_starts = df.index[(df["USRECP"] == 1) & (df["USRECP"].shift(1) == 0)]

    # Find the end of a recession (where USREC goes from 1 to 0)
    # We need to look at the day *before* the switch to 0
    recession_ends = df.index[(df["USRECP"] == 0) & (df["USRECP"].shift(1) == 1)]

    # Handle edge cases: if the data starts or ends during a recession
    if df["USRECP"].iloc[0] == 1:
        recession_starts = recession_starts.insert(0, df.index[0])
    if df["USRECP"].iloc[-1] == 1:
        recession_ends = recession_ends.append(pd.Index([df.index[-1]]))

    # Add a shaded rectangle for each recession period found
    for start, end in zip(recession_starts, recession_ends):
        fig.add_vrect(
            x0=start,
            x1=end,
            fillcolor="grey",
            opacity=0.2,
            layer="below",
            line_width=0,
        )

# COVID-19 specific recession period
covid_start = pd.Timestamp("2020-02-01")
covid_end = pd.Timestamp("2020-04-30")
fig.add_vrect(
    x0=covid_start,
    x1=covid_end,
    fillcolor="grey",
    opacity=0.2,
    layer="below",
    line_width=0,
    annotation_text="<b>COVID-19 Breakout</b>",
    annotation_position="top left",
)


# -- Customize the Layout --
fig.update_layout(
    title_text="<b>Systemic Risk Index vs. S&P 500 Performance</b><br><sup>SRI spikes during NBER-defined recessions</sup>",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="right", x=1),
    # Set y-axis titles
    yaxis=dict(
        title="<b>Systemic Risk Index (0-100)</b>",
        tickfont=dict(color="crimson"),
        range=[0, 105],
        showgrid=False,
    ),
    yaxis2=dict(
        title="<b>S&P 500 Price (USD)</b>",
        # titlefont=dict(color="royalblue"),
        tickfont=dict(color="royalblue"),
        overlaying="y",
        side="right",
    ),
    xaxis=dict(
        showgrid=False,
        tickformat="%Y-%m",
        ticks="outside",
    ),
    xaxis_title="Date",
    autosize=False,
    width=1050,
    height=600,
)

fig.add_annotation(
    x=0,
    y=1.02,
    xanchor="left",
    yanchor="bottom",
    xref="paper",
    yref="paper",
    showarrow=False,
    align="left",
    text="<b>NBER based Recession</b>",
    bgcolor="rgba(128,128,128,0.3)",
)

fig.show()

### Analysis: SRI vs. S&P 500

The chart above confirms that our Systemic Risk Index effectively captures periods of significant market stress.

**Key Observations:**

*   **Inverse Relationship:** The SRI (in crimson) exhibits a strong inverse relationship with the S&P 500 (in blue). As the SRI spikes, the SPY experiences sharp declines.
*   **Crisis Alignment:** The most prominent spikes in the SRI align perfectly with the 2008 Global Financial Crisis and the 2020 COVID-19 market crash, both of which correspond to the shaded periods.
*   **Early Warning Potential:** The index often begins its ascent before the market's sharpest sell-off, particularly noticeable in the lead-up to the 2008 crisis. This suggests its potential as an early-warning indicator.

## Plot 2: 52-Week Rolling Correlation of SRI with Key Assets

Static, long-term correlations can be misleading as relationships between assets change, especially during crises. To analyze this dynamic behavior, we calculate the 52-week rolling correlation between the weekly returns of SPY, GLD, and TLT and our Systemic Risk Index.

A value approaching **-1.0** indicates that as risk rises, the asset's price tends to fall (risk-on behavior). A value approaching **+1.0** suggests that as risk rises, the asset's price also tends to rise (safe-haven behavior).

In [ ]:
# Create a dataframe for 'SPY', 'GLD' & 'TLT'
df_assets = df[["SPY", "GLD", "TLT"]].copy()

# Calculate weekly log returns
df_assets = df_assets.apply(np.log).diff()

# Calculate the 26-week rolling correlation between 'SRI' and each asset
rolling_corr = df_assets.rolling(window=52).corr(df["SRI"])

# Display the rolling correlations with a line chart
fig = make_subplots(
    rows=3,
    shared_xaxes=True,
    shared_yaxes=True,
    subplot_titles=("SPY vs SRI", "GLD vs SRI", "TLT vs SRI"),
)

fig.add_trace(
    go.Scatter(
        x=rolling_corr.index,
        y=rolling_corr["SPY"],
        name="SPY",
        line=dict(color="royalblue", width=2),
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=rolling_corr.index,
        y=rolling_corr["GLD"],
        name="GLD",
        line=dict(color="crimson", width=2),
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=rolling_corr.index,
        y=rolling_corr["TLT"],
        name="TLT",
        line=dict(color="darkorange", width=2),
    ),
    row=3,
    col=1,
)

if df["USRECP"].sum() > 0:
    # Find the start of a recession (where USREC goes from 0 to 1)
    recession_starts = df.index[(df["USRECP"] == 1) & (df["USRECP"].shift(1) == 0)]

    # Find the end of a recession (where USREC goes from 1 to 0)
    # We need to look at the day *before* the switch to 0
    recession_ends = df.index[(df["USRECP"] == 0) & (df["USRECP"].shift(1) == 1)]

    # Handle edge cases: if the data starts or ends during a recession
    if df["USRECP"].iloc[0] == 1:
        recession_starts = recession_starts.insert(0, df.index[0])
    if df["USRECP"].iloc[-1] == 1:
        recession_ends = recession_ends.append(pd.Index([df.index[-1]]))

    # Add a shaded rectangle for each recession period found
    for start, end in zip(recession_starts, recession_ends):
        fig.add_vrect(
            x0=start,
            x1=end,
            fillcolor="grey",
            opacity=0.2,
            layer="below",
            line_width=0,
            annotation_text="<b>NBER based Recession</b>",
            annotation_position="top left",
        )

# COVID-19 specific recession period
covid_start = pd.Timestamp("2020-02-01")
covid_end = pd.Timestamp("2020-04-30")
fig.add_vrect(
    x0=covid_start,
    x1=covid_end,
    fillcolor="grey",
    opacity=0.2,
    layer="below",
    line_width=0,
    annotation_text="<b>COVID-19 Breakout</b>",
    annotation_position="top left",
)

fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="grey")

fig.update_layout(
    title_text="<b>52 Week Rolling Correlation of 'SPY', 'GLD' & 'TLT' with SRI</b>",
    template="plotly_white",
    height=800,
    showlegend=False,
)

fig.update_yaxes(title_text="Correlation", showgrid=False)

fig.update_xaxes(showgrid=False)

fig.show()

### Analysis: Dynamic Correlations Reveal Shifting Asset Roles

This series of plots illustrates the 52-week rolling correlation between the Systemic Risk Index (SRI) and the weekly returns of SPY, GLD, and TLT. This analysis is crucial for understanding how asset relationships change, especially during crises.

**Key Observations:**

*   **SPY vs. SRI (Risk-On Asset):** The correlation is consistently negative. During crises, it becomes strongly negative (approaching -0.8 in 2020), confirming that as systemic risk spikes, equity markets sell off. This is the expected behavior of a risk-on asset.

*   **GLD vs. SRI (Complex Safe Haven):** Gold's role is complex. It acted as a safe haven during the 2008 crisis, with its correlation turning positive. However, during the initial COVID-19 shock in March 2020, **the correlation plunged sharply into negative territory.** This indicates that in an acute liquidity crisis, gold was sold alongside risk assets as investors scrambled for cash.

*   **TLT vs. SRI (Reliable Safe Haven):** Long-term Treasury bonds have proven to be a more consistent safe haven. The correlation with the SRI became strongly positive during both the 2008 and 2020 crises. This demonstrates a reliable "flight-to-safety" effect where investors buy government bonds when systemic risk is highest.

## Plot 3: Adding the U.S. Dollar to the Rolling Correlation Analysis

To confirm the "dash for cash" hypothesis observed in gold's behavior during the COVID-19 crisis, we now add the U.S. Dollar Bullish Fund (UUP) to our rolling correlation analysis. If the theory holds, we expect to see a strong positive correlation between the SRI and UUP during COVID-19 shock in March 2020, as investors flee all other asset classes for the liquidity of the world's reserve currency.

In [ ]:
# Create a dataframe for all four assets
df_assets_all = df[["SPY", "GLD", "TLT", "UUP"]].copy()

# Calculate weekly log returns
df_assets_all = df_assets_all.apply(np.log).diff()

# Calculate the 52-week rolling correlation with SRI
rolling_corr_all = df_assets_all.rolling(window=52).corr(df["SRI"])

# Create a 4-panel subplot
fig = make_subplots(
    rows=4,
    shared_xaxes=True,
    shared_yaxes=True,
    subplot_titles=("SPY vs SRI", "GLD vs SRI", "TLT vs SRI", "UUP vs SRI"),
)

# Add SPY trace
fig.add_trace(
    go.Scatter(
        x=rolling_corr_all.index,
        y=rolling_corr_all["SPY"],
        name="SPY",
        line=dict(color="royalblue", width=2),
    ),
    row=1,
    col=1,
)

# Add GLD trace
fig.add_trace(
    go.Scatter(
        x=rolling_corr_all.index,
        y=rolling_corr_all["GLD"],
        name="GLD",
        line=dict(color="crimson", width=2),
    ),
    row=2,
    col=1,
)

# Add TLT trace
fig.add_trace(
    go.Scatter(
        x=rolling_corr_all.index,
        y=rolling_corr_all["TLT"],
        name="TLT",
        line=dict(color="darkorange", width=2),
    ),
    row=3,
    col=1,
)

# Add UUP trace
fig.add_trace(
    go.Scatter(
        x=rolling_corr_all.index,
        y=rolling_corr_all["UUP"],
        name="UUP",
        line=dict(color="green", width=2),
    ),
    row=4,
    col=1,
)

if df["USRECP"].sum() > 0:
    # Find the start of a recession (where USREC goes from 0 to 1)
    recession_starts = df.index[(df["USRECP"] == 1) & (df["USRECP"].shift(1) == 0)]

    # Find the end of a recession (where USREC goes from 1 to 0)
    # We need to look at the day *before* the switch to 0
    recession_ends = df.index[(df["USRECP"] == 0) & (df["USRECP"].shift(1) == 1)]

    # Handle edge cases: if the data starts or ends during a recession
    if df["USRECP"].iloc[0] == 1:
        recession_starts = recession_starts.insert(0, df.index[0])
    if df["USRECP"].iloc[-1] == 1:
        recession_ends = recession_ends.append(pd.Index([df.index[-1]]))

    # Add a shaded rectangle for each recession period found
    for start, end in zip(recession_starts, recession_ends):
        fig.add_vrect(
            x0=start,
            x1=end,
            fillcolor="grey",
            opacity=0.2,
            layer="below",
            line_width=0,
            annotation_text="<b>NBER based Recession</b>",
            annotation_position="top left",
        )

# Add March 2020 vertical line
march = pd.Timestamp("2020-03-20")
fig.add_vline(x=march, line_width=2, line_dash="dash", line_color="red")

# Add March 2020 annotation
fig.add_annotation(
    x=march,
    y=1,
    xref="x",
    yref="paper",
    text="<b>March 2020</b>",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    font=dict(size=12, color="red"),
    bgcolor="rgba(255,255,255,0.7)",
)

# Add horizontal line at y=0 for all subplots
for i in range(1, 5):
    fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey", row=i, col=1)

# Update layout
fig.update_layout(
    title_text="<b>52-Week Rolling Correlation of Assets with SRI (UUP included)</b>",
    template="plotly_white",
    height=900,
    showlegend=False,
)

fig.update_yaxes(title_text="Correlation", showgrid=False)
fig.update_xaxes(showgrid=False)

fig.show()

### Analysis: The March 2020 "Dash for Cash" Visualized

This series of rolling correlations provides a powerful visualization of asset behavior during market stress, with the COVID-19 shock in March 2020 serving as a perfect case study.

**Key Observations from the March 2020 Crisis:**

*   **UUP vs. SRI (Flight to Liquidity):** As the crisis unfolded, the correlation between the U.S. Dollar (UUP) and the Systemic Risk Index (SRI) spiked dramatically, reaching a peak of approximately **+0.4**. This indicates a strong "flight to liquidity," where investors abandoned other assets and flocked to the U.S. dollar as the ultimate safe haven, causing its value to rise in lockstep with systemic risk.

*   **GLD vs. SRI (Liquidation Event):** Gold's correlation, which had been positive, sharply inverted to nearly **-0.4**. The soaring demand for U.S. dollars forced investors to liquidate their most liquid and profitable positions to raise cash. Gold, having rallied into the crisis, became a source of funds. It was sold not because it was perceived as risky, but because it was a liquid asset that could be quickly converted to cash.

*   **SPY vs. SRI (Risk-Off):** As expected, the correlation between equities (SPY) and the SRI plunged to its most negative point (below -0.75), demonstrating a classic risk-off move where rising systemic risk triggered a severe stock market sell-off.

*   **TLT vs. SRI (Initial Haven Behavior):** Treasury bonds (TLT) initially behaved as a safe haven, with their correlation briefly spiking. However, it quickly dropped close to zero during the peak of the liquidity crunch, suggesting that even these safe assets were sold off to meet margin calls and raise cash before central bank intervention stabilized the market.

**Conclusion:**

The March 2020 period vividly illustrates the "dash for cash" phenomenon. The simultaneous negative turn in the correlations for SPY and GLD, coupled with the sharp positive spike in the UUP correlation, confirms that the market panic was primarily a liquidity crisis. This analysis underscores that during the most acute phases of a crisis, the U.S. dollar's role as the ultimate safe haven can temporarily override the traditional behavior of other assets like gold.